In [2]:
import warnings
from pathlib import Path
from loguru import logger
from datetime import datetime
from typing import Optional, Union
from itertools import chain

from astropy.io import fits
from astropy.io import votable
from astropy.io.votable import tree

import astropy.wcs as wcs
from astropy.coordinates import SkyCoord

import astropy.units as u
from astropy.nddata.utils import Cutout2D
from astropy.time import Time
from mocpy import MOC, STMOC

from vast_post_processing import crop, corrections

DATA_PATH = (Path(".") / "test-data" / "TILES").resolve()
IMAGE_PATH = (
    DATA_PATH
    / "STOKESI_IMAGES"
    / "epoch_38"
    / "image.i.VAST_0334-37.SB50801.cont.taylor.0.restored.conv.fits"
)
VOTABLE_PATH = (
    DATA_PATH
    / "STOKESI_SELAVY"
    / "epoch_38"
    / "selavy-image.i.VAST_0334-37.SB50801.cont.taylor.0.restored.conv.components.xml"
)

In [3]:
hdu = fits.open(IMAGE_PATH)[0]
field_centre = crop.get_field_centre(hdu.header)
cropped_hdu = crop.crop_hdu(hdu, field_centre)
vot = votable.parse(VOTABLE_PATH)
cropped_table = crop.crop_catalogue(vot, cropped_hdu)

2023-08-23 12:32:14.629 | DEBUG    | vast_post_processing.crop:get_field_centre:28 - Finding field centre
/home/hansen/.cache/pypoetry/virtualenvs/vast-post-processing-SFvl3Fo4-py3.9/lib/python3.9/site-packages/astropy/wcs/wcs.py:819: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to   116.658800 from OBSGEO-[XYZ].
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'.
  warnings.warn(
2023-08-23 12:32:14.649 | DEBUG    | vast_post_processing.crop:get_field_centre:35 - <SkyCoord (FK5: equinox=2000.0): (ra, dec) in deg
    (53.61702083, -37.63907778)>
2023-08-23 12:32:14.651 | DEBUG    | vast_post_processing.crop:crop_hdu:49 - Cropping HDU


2023-08-23 12:32:15.492 | DEBUG    | vast_post_processing.crop:crop_catalogue:72 - Cropping catalogue
/home/hansen/.cache/pypoetry/virtualenvs/vast-post-processing-SFvl3Fo4-py3.9/lib/python3.9/site-packages/astropy/wcs/wcs.py:819: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to   116.658800 from OBSGEO-[XYZ].
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'.
  warnings.warn(


In [4]:
moc = crop.wcs_to_moc(cropped_hdu)
stmoc = crop.moc_to_stmoc(moc, hdu)
print(stmoc)

2023-08-23 12:32:15.716 | DEBUG    | vast_post_processing.crop:wcs_to_moc:87 - Creating MOC
/home/hansen/.cache/pypoetry/virtualenvs/vast-post-processing-SFvl3Fo4-py3.9/lib/python3.9/site-packages/astropy/wcs/wcs.py:819: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to   116.658800 from OBSGEO-[XYZ].
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'.
  warnings.warn(


KeyError: "Keyword 'DATE-BEG' not found."

In [11]:
from uncertainties import ufloat
from astropy.coordinates import Angle
from vast_post_processing.catalogs import Catalog
from vast_post_processing.crossmatch import (
    crossmatch_qtables,
    calculate_positional_offsets,
    calculate_flux_offsets,
)

radius = Angle("10arcsec")
condon = False
psf_reference = None
psf = None
fix_m = False
fix_b = False
positional_unit = u.Unit("arcsec")
flux_unit = u.Unit("mJy")
crossmatch_output = None
csv_output = None

reference_catalog_path = VOTABLE_PATH
catalog_path = VOTABLE_PATH
flux_unit /= u.beam  # add beam divisor as we currently only work with peak fluxes

# Create Catalog objects
reference_catalog = Catalog(
    reference_catalog_path,
    psf=psf_reference,
    condon=condon,
    input_format="selavy",
)
catalog = Catalog(
    catalog_path,
    psf=psf,
    condon=condon,
    input_format="selavy",
)

# Perform the crossmatch
xmatch_qt = crossmatch_qtables(catalog, reference_catalog, radius=radius)

# Select xmatches with non-zero flux errors and no siblings
mask = xmatch_qt["flux_peak_err"] > 0
mask &= xmatch_qt["flux_peak_err_reference"] > 0
mask &= xmatch_qt["has_siblings"] == 0
mask &= xmatch_qt["has_siblings_reference"] == 0
data = xmatch_qt[mask]

# Write the crossmatch data to csv
if crossmatch_output is not None:
    data.write(crossmatch_output, overwrite=True)

# Calculate positional offsets
dra_median, ddec_median, dra_madfm, ddec_madfm = calculate_positional_offsets(data)
dra_median_value = dra_median.to(positional_unit).value
dra_madfm_value = dra_madfm.to(positional_unit).value
ddec_median_value = ddec_median.to(positional_unit).value
ddec_madfm_value = ddec_madfm.to(positional_unit).value

# Calculate flux ratio
gradient, offset, gradient_err, offset_err = calculate_flux_offsets(
    data, fix_m=fix_m, fix_b=fix_b
)
ugradient = ufloat(gradient, gradient_err)
uoffset = ufloat(offset.to(flux_unit).value, offset_err.to(flux_unit).value)

flux_corr_mult = 1 / ugradient
flux_corr_add = -1 * uoffset

print(type(dra_median_value), dra_median_value)
print(type(ddec_median_value), ddec_median_value)
print(type(flux_corr_mult), flux_corr_mult)
print(type(flux_corr_add), flux_corr_add)

2023-08-23 12:47:32.147 | DEBUG    | vast_post_processing.catalogs:_read_catalog:273 - Reading /home/hansen/projects/vast/projects/vast-post-processing/tests/test-data/TILES/STOKESI_SELAVY/epoch_38/selavy-image.i.VAST_0334-37.SB50801.cont.taylor.0.restored.conv.components.xml as a Selavy VOTable catalog.
2023-08-23 12:47:32.781 | INFO     | vast_post_processing.catalogs:_filter_sources:299 - Filtering 0 sources with fluxes <= 0
2023-08-23 12:47:32.790 | INFO     | vast_post_processing.catalogs:_filter_sources:305 - Filtering 0 sources with fitted sizes <= 0.
2023-08-23 12:47:32.800 | INFO     | vast_post_processing.catalogs:_filter_sources:314 - Filtering 28 sources that are not point sources.
2023-08-23 12:47:32.818 | INFO     | vast_post_processing.catalogs:_filter_sources:323 - Filtering 4628 sources with SNR <= 20
2023-08-23 12:47:32.838 | INFO     | vast_post_processing.catalogs:_filter_sources:329 - Filtering 2020 sources that have neighbors within 1 arcmin.
2023-08-23 12:47:32.8

<class 'astropy.utils.masked.core.MaskedNDArray'> 0.
<class 'astropy.utils.masked.core.MaskedNDArray'> 0.
<class 'uncertainties.core.AffineScalarFunc'> 1.0+/-0
<class 'uncertainties.core.AffineScalarFunc'> -0.0+/-0


TypeError: isinstance() arg 2 must be a type or tuple of types

In [21]:
import uncertainties
import numpy as np

print(uncertainties.core.__all__)

['ufloat', 'ufloat_fromstr', 'nominal_value', 'std_dev', 'covariance_matrix', 'UFloat', 'wrap', 'correlated_values', 'correlated_values_norm', 'correlation_matrix']
